In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp

In [127]:
df = pd.read_csv('face feature vectors.csv').drop('Unnamed: 0', axis=1)
#rename first column
df.rename(columns={'Unnamed: 1':'Gender'}, inplace=True)
print(df)

     Gender         0         1         2         3         4         5  \
0      male -0.066420  0.151611  0.027740  0.052771 -0.066105 -0.041232   
1      male -0.030614  0.049667  0.008084 -0.050324  0.007649 -0.063818   
2      male -0.096178  0.061127  0.035326 -0.035388 -0.090728 -0.018634   
3      male -0.103057  0.085044  0.078333 -0.035873 -0.028163  0.004924   
4      male -0.125815  0.120046  0.023131 -0.042901  0.038215 -0.049677   
..      ...       ...       ...       ...       ...       ...       ...   
795  female -0.164731  0.064301  0.058630 -0.017420 -0.157600 -0.022536   
796  female -0.095308  0.051095  0.092913 -0.101745 -0.083153 -0.028159   
797  female -0.202852  0.037039  0.079731 -0.047156 -0.140062 -0.080246   
798  female -0.088300  0.063530  0.049627 -0.026011 -0.172773  0.086218   
799  female -0.156201  0.055165  0.142716 -0.115393 -0.128982 -0.139830   

            6         7         8  ...       118       119       120  \
0   -0.002637 -0.158467  0.

In [128]:
classes = df['Gender'].unique()
train = pd.DataFrame()
test = pd.DataFrame()
for c in classes:
    train = pd.concat([train, df[df['Gender'] == c].iloc[5:]], ignore_index=True)
    test = pd.concat([test, df[df['Gender'] == c].iloc[:5]], ignore_index=True)

print(train)

     Gender         0         1         2         3         4         5  \
0      male -0.149119  0.125288  0.142323 -0.009087 -0.031394 -0.123533   
1      male -0.139035  0.073513 -0.001770 -0.034225 -0.101610  0.065105   
2      male -0.074126 -0.000669  0.004166 -0.082413 -0.096091 -0.021992   
3      male -0.166220  0.042769 -0.031647 -0.036892 -0.143837 -0.040566   
4      male -0.185770  0.154008  0.073184 -0.070829 -0.144617 -0.019732   
..      ...       ...       ...       ...       ...       ...       ...   
785  female -0.164731  0.064301  0.058630 -0.017420 -0.157600 -0.022536   
786  female -0.095308  0.051095  0.092913 -0.101745 -0.083153 -0.028159   
787  female -0.202852  0.037039  0.079731 -0.047156 -0.140062 -0.080246   
788  female -0.088300  0.063530  0.049627 -0.026011 -0.172773  0.086218   
789  female -0.156201  0.055165  0.142716 -0.115393 -0.128982 -0.139830   

            6         7         8  ...       118       119       120  \
0    0.043598 -0.063999  0.

In [129]:
print(test)

   Gender         0         1         2         3         4         5  \
0    male -0.066420  0.151611  0.027740  0.052771 -0.066105 -0.041232   
1    male -0.030614  0.049667  0.008084 -0.050324  0.007649 -0.063818   
2    male -0.096178  0.061127  0.035326 -0.035388 -0.090728 -0.018634   
3    male -0.103057  0.085044  0.078333 -0.035873 -0.028163  0.004924   
4    male -0.125815  0.120046  0.023131 -0.042901  0.038215 -0.049677   
5  female  0.001747  0.185678  0.073260  0.042142 -0.088674  0.028186   
6  female -0.091598  0.095340  0.072125 -0.092276 -0.079953  0.047782   
7  female -0.018751  0.088572  0.068894 -0.065700 -0.115126  0.024339   
8  female -0.130889  0.093262  0.122244 -0.110014 -0.157625 -0.036781   
9  female -0.037433  0.078158  0.118061 -0.117658 -0.194807 -0.045464   

          6         7         8  ...       118       119       120       121  \
0 -0.002637 -0.158467  0.130467  ...  0.025989 -0.001087  0.027260 -0.046754   
1 -0.019530 -0.119905  0.186553  ...

In [135]:
def fit_Bayes_classifier(train):
    classes = train['Gender'].unique()
    mean_vector = {}
    cov_matrix = {}
    inv_cov_matrix = {}
    det_cov_matrix = {}

    for c in classes:
        mean_vector[c] = np.array(train[train['Gender'] == c].iloc[:, 1:].mean())
        cov_matrix[c] = np.array(train[train['Gender'] == c].iloc[:, 1:].cov())
        inv_cov_matrix[c] = np.linalg.inv(cov_matrix[c])
        det_cov_matrix[c] = sp.Matrix(cov_matrix[c]).det()
        print(det_cov_matrix[c])

    return mean_vector, inv_cov_matrix, det_cov_matrix


def Bayes_classifier(train, test_vector, mean_vector, inv_cov_matrix, det_cov_matrix):
    classes = train['Gender'].unique()
    dim = len(train.columns) - 1
    bayes_prob = []

    for c in classes:
        apriori = len(train[train['Gender'] == c]) / len(train)
        test_vector = np.array(test_vector)
        prob = np.exp(-0.5 * np.dot(np.dot((test_vector - mean_vector[c]).T, inv_cov_matrix[c]), (test_vector - mean_vector[c]))) / (
                    np.power(2 * np.pi, dim / 2) * np.power(det_cov_matrix[c], 0.5))
        bayes_prob.append(prob * apriori)

    return classes[np.argmax(bayes_prob)]
        
        

In [137]:
mean_vector, inv_cov_matrix, det_cov_matrix = fit_Bayes_classifier(train)

8.51759027713774e-863
4.77521343772887e-864


In [138]:
predictions = []
for i in range(len(test)):
    predictions.append(Bayes_classifier(train, test.iloc[i, 1:], mean_vector, inv_cov_matrix, det_cov_matrix))

compare = pd.DataFrame({'Actual': test['Gender'], 'Predicted': predictions})

print(compare)

   Actual Predicted
0    male      male
1    male      male
2    male      male
3    male      male
4    male      male
5  female      male
6  female    female
7  female    female
8  female    female
9  female    female


In [139]:
def accuracy(y_pred,y_test):
    return np.sum(y_pred == y_test)/len(y_test)

print(accuracy(predictions, test['Gender']))

0.9
